In [125]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import h5py as h5
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
from pandas.api.types import is_numeric_dtype

In [126]:
KFOLD = 5
EXP_RESULTS_DIR = "../results/timewarp_aug"
SPLITS = range(KFOLD)
EVALS = [{"train_dataset":"instance", "test_dataset":"instance", "model_name":"AugmentationCrossCovarianceMonitor30s", "epoch":10, "monitor_params":["fcov"]},
         {"train_dataset":"instance", "test_dataset":"stead", "model_name":"AugmentationCrossCovarianceMonitor30s", "epoch":10, "monitor_params":["fcov"]},
         {"train_dataset":"stead", "test_dataset":"instance", "model_name":"AugmentationCrossCovarianceMonitor30s", "epoch": 10, "monitor_params":["fcov"]},
         {"train_dataset":"stead", "test_dataset":"stead", "model_name":"AugmentationCrossCovarianceMonitor30s", "epoch": 10, "monitor_params":["fcov"]}]

In [127]:
def get_trace_path(split_idx, train_dataset, test_dataset, model_name):
    return "{}/training_{}/testing_{}/{}/split{}/traces.csv".format(EXP_RESULTS_DIR, train_dataset, test_dataset, model_name, split_idx)

def get_monitor_path(split_idx, train_dataset, test_dataset, model_name, epoch, monitor_params):
    return "{}/training_{}/testing_{}/{}/split{}/epoch{}_monitoredparams{}.hdf5".format(EXP_RESULTS_DIR, train_dataset, test_dataset, model_name, split_idx, epoch, monitor_params)

In [128]:
def softmax(fc, gain=4, axis=1):
    # First normalize the fc values.
    # Make it's standart deviation 1. And mean 0.
    fc = fc - np.mean(fc, axis=axis, keepdims=True)
    fc = fc / (1e-37+np.std(fc, axis=axis, keepdims=True))

    fc = fc - np.max(fc, axis=axis, keepdims=True)
    return np.exp(gain * fc) / np.sum(np.exp(gain * fc), axis=axis, keepdims=True)

def linear_map(x, a, b):
    return a + (b - a) * (x - np.min(x)) / (np.max(x) - np.min(x))

def gaussian(x, mu, sig, axis=1):
    g = np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))
    return g / np.sum(g, axis=axis, keepdims=True)

def detection_probability_ensemble(cov, window_size=5.0):
    # (BATCH_SIZE, N_ENSEMBLE, N_TIMESTEPS)
    ts = np.shape(cov)[2]
    t = np.expand_dims(np.expand_dims(np.linspace(-15, 15, ts), axis=0), axis=0)

    g = gaussian(t, np.zeros_like(t), window_size*np.ones_like(t), axis=2)

    return np.mean(np.sum(cov * g, axis=(2)), axis=1)

def detection_probability(cov, window_size=5.0):
    ts = np.shape(cov)[1]
    t = np.expand_dims(np.linspace(-15, 15, ts), axis=0)
    
    g = gaussian(t, np.zeros_like(t), window_size*np.ones_like(t))
    
    return np.sum(cov * g, axis=1)

def batch_normalize_feature(f):
    # Demean each channel over time axes.
    f = f - np.mean(f, axis=(1), keepdims=True)
    
    # Normalize each channel by dividing it to standart deviation.
    f = f / np.sqrt(np.mean(np.square(f), axis=(0, 1), keepdims=True))

    return f

def get_maxf1_threshold(thresholds, pre, rec):
    f1_score = 2 * pre * rec / (1e-37 + pre + rec)
    max_f1_score_idx = np.argmax(f1_score)
    return thresholds[max_f1_score_idx]

def get_balanced_accuracy(tpr, fpr, thresholds, best_f1_threshold):
    idx = np.argmin(np.abs(thresholds - best_f1_threshold))
    return ((tpr[idx] + (1 - fpr[idx])) / 2)

def get_metrics(pre, rec, thresholds, best_f1_threshold):
    idx = np.argmin(np.abs(thresholds - best_f1_threshold))
    f1_score = 2 * pre * rec / (1e-37 + pre + rec)
    return {
        "pre":pre[idx],
        "rec":rec[idx],
        "f1_score":f1_score[idx]}
    
def plot_density(df_traces, metric):
    plt.clf()
    plt.figure(figsize=(10, 8))
    sns.kdeplot(data=df_traces, hue="label", x=df_traces[metric], bw_adjust=0.05)

def plot_roc(models):
    plt.clf()
    plt.figure(figsize=(10, 8))
    ax = plt.subplot(111)
    ax.set_xlabel("FPR")
    ax.set_ylabel("TPR")

    for model in models:
        ax.plot(model["fpr"],  model["tpr"], label =model["name"])
    
    ax.legend()

def is_test_sample(df):
    df["min_crop_offset"] = df["p_arrival_sample"] + 300 - 3000
    df["max_crop_offset"] = df["p_arrival_sample"] - 300

    return ((df["min_crop_offset"] < df["crop_offset"]) & ((df["crop_offset"] < df["max_crop_offset"])))

def get_latex_table(df_metrics):
    columns = ["Method", "Training", "Testing", "ROC-AUC", "Precision", "Recall", "F1-Score"]
    trainsets = df_metrics["training"].unique()
    testsets = df_metrics["testing"].unique()
    models = df_metrics["model"].unique()

    latex_table = ""
    for column in columns:
        latex_table += "& {} ".format(column)
    latex_table += "\\\\ \\hline \n"

    for trainset in trainsets:
        for testset in testsets:
            for model in models:
                df_exp = df_metrics[(df_metrics["model"] == model) & (df_metrics["training"] == trainset) & (df_metrics["testing"] == testset)]
                latex_table += "\\\\ \\hline \n"
                for column in columns:
                    if is_numeric_dtype(df_exp[column]):
                        latex_table += "& ${:.3f} \pm {:.3f}$ ".format(np.mean(df_exp[column]), np.std(df_exp[column]))
                    else:
                        latex_table += "& {} ".format(df_exp[column].values[0])
                latex_table += "\\\\ \\hline \n"
    
    return latex_table

In [129]:
df_list = []
fc_list = []
df_metrics_list = []
for eval in EVALS:    
    for i in range(KFOLD):
        with h5.File(get_monitor_path(split_idx=i, 
                                      train_dataset=eval["train_dataset"], 
                                      test_dataset=eval["test_dataset"], 
                                      model_name=eval["model_name"],
                                      epoch=eval["epoch"],
                                      monitor_params=eval["monitor_params"]), "r") as f:
            
            df_path = get_trace_path(split_idx=i, 
                           train_dataset = eval["train_dataset"], 
                           test_dataset=eval["test_dataset"], 
                           model_name = eval["model_name"])


            df_traces = pd.read_csv(df_path)

            fcov = np.array(f["fcov"])
            print("Total nans", np.isnan(fcov).sum())

            # Reset traces index.
            df_traces.reset_index(inplace=True)
            df_eq = df_traces[df_traces.label == "eq"]
            df_no = df_traces[df_traces.label == "no"]

            # Eliminate ill cropped traces.
            df_eq = df_eq[is_test_sample(df_eq)]

            # Concat eq and no traces. And shuffle them. Index.
            df_traces = pd.concat([df_eq, df_no], ignore_index=False)
            df_traces = df_traces.sample(frac=1)
            fcov = fcov[np.array(df_traces.index), :]

            df_list.append(df_traces)
            fc_list.append(fcov)

            #df_traces["similarity"]= detection_probability_ensemble(fcov)
            df_traces["similarity"] = detection_probability(fcov)
            
            fpr, tpr, thresholds1 = roc_curve(df_traces["label"] == "eq", df_traces["similarity"])
            pre, rec, thresholds2 = precision_recall_curve(df_traces["label"] == "eq", df_traces["similarity"])
            roc_auc = auc(fpr, tpr)
            av_pre = average_precision_score(df_traces["label"] == "eq", df_traces["similarity"])

            best_f1_threshold = get_maxf1_threshold(thresholds2, pre, rec)
            metrics = get_metrics(pre, rec, thresholds2, best_f1_threshold)
            metrics.update({"model":eval["model_name"], "training":eval["train_dataset"], "testing":eval["test_dataset"], "auc":roc_auc, "av_pre":av_pre})
            df_metrics_list.append(metrics)

df_metrics = pd.DataFrame(df_metrics_list)
print("Results:")
print(get_latex_table(df_metrics))


Total nans 0


/tmp/ipykernel_316938/1220699846.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["min_crop_offset"] = df["p_arrival_sample"] + 300 - 3000
/tmp/ipykernel_316938/1220699846.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["max_crop_offset"] = df["p_arrival_sample"] - 300


Total nans 0


/tmp/ipykernel_316938/1220699846.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["min_crop_offset"] = df["p_arrival_sample"] + 300 - 3000
/tmp/ipykernel_316938/1220699846.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["max_crop_offset"] = df["p_arrival_sample"] - 300


Total nans 0


...

Total nans 0


/tmp/ipykernel_316938/1220699846.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["min_crop_offset"] = df["p_arrival_sample"] + 300 - 3000
/tmp/ipykernel_316938/1220699846.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["max_crop_offset"] = df["p_arrival_sample"] - 300
/tmp/ipykernel_316938/15461894.py:19: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_traces = pd.read_csv(df_path)


Total nans 0


/tmp/ipykernel_316938/1220699846.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["min_crop_offset"] = df["p_arrival_sample"] + 300 - 3000
/tmp/ipykernel_316938/1220699846.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["max_crop_offset"] = df["p_arrival_sample"] - 300


Results:
& model & training & testing & auc & pre & rec & f1_score \\ \hline 
& AugmentationCrossCovarianceMonitor30s & instance & instance & $0.963 \pm 0.006$ & $0.964 \pm 0.002$ & $0.961 \pm 0.002$ & $0.963 \pm 0.002$ \\ \hline 
& model & training & testing & auc & pre & rec & f1_score \\ \hline 
& AugmentationCrossCovarianceMonitor30s & instance & stead & $0.969 \pm 0.013$ & $0.933 \pm 0.011$ & $0.967 \pm 0.003$ & $0.950 \pm 0.006$ \\ \hline 
& model & training & testing & auc & pre & rec & f1_score \\ \hline 
& AugmentationCrossCovarianceMonitor30s & stead & instance & $0.966 \pm 0.003$ & $0.965 \pm 0.002$ & $0.960 \pm 0.002$ & $0.963 \pm 0.002$ \\ \hline 
& model & training & testing & auc & pre & rec & f1_score \\ \hline 
& AugmentationCrossCovarianceMonitor30s & stead & stead & $0.972 \pm 0.007$ & $0.936 \pm 0.009$ & $0.969 \pm 0.002$ & $0.952 \pm 0.005$ \\ \hline 



In [22]:

df_metrics = pd.DataFrame(df_metrics_list)
auc_std = np.std(df_metrics["auc"])
acc_std = np.std(df_metrics["av_pre"])
pre_std = np.std(df_metrics["pre"])
rec_std = np.std(df_metrics["rec"])
f1_score_std = np.std(df_metrics["f1_score"])

print(df_metrics)
print("auc: {:.3f} +- {:.3f}".format(np.mean(df_metrics["auc"]), auc_std))
print("av_pre: {:.3f} +- {:.3f}".format(np.mean(df_metrics["av_pre"]), acc_std))
print("pre: {:.3f} +- {:.3f}".format(np.mean(df_metrics["pre"]), pre_std))
print("rec: {:.3f} +- {:.3f}".format(np.mean(df_metrics["rec"]), rec_std))
print("f1_score: {:.3f} +- {:.3f}".format(np.mean(df_metrics["f1_score"]), f1_score_std))

        pre       rec  f1_score       auc    av_pre
0  0.946742  0.970300  0.958376  0.984501  0.984999
1  0.947168  0.969765  0.958333  0.985403  0.986940
2  0.946564  0.970925  0.958590  0.984865  0.985228
3  0.953315  0.974105  0.963598  0.987495  0.987972
4  0.938192  0.971336  0.954476  0.982033  0.981282
auc: 0.985 +- 0.002
av_pre: 0.985 +- 0.002
pre: 0.946 +- 0.005
rec: 0.971 +- 0.002
f1_score: 0.959 +- 0.003


In [23]:
print(get_latex_table(df_metrics))

& auc & av_pre & pre & rec & f1_score \\ \hline 
& $0.985 \pm 0.002$ & $0.985 \pm 0.002$ & $0.946 \pm 0.005$ & $0.971 \pm 0.002$ & $0.959 \pm 0.003$ \\ \hline 



In [24]:
print(get_latex_table_kfold(df_metrics))

& auc & av_pre & pre & rec & f1_score \\ \hline 
& $0.985 & $0.985 & $0.947 & $0.970 & $0.958 \\ \hline 
& $0.985 & $0.987 & $0.947 & $0.970 & $0.958 \\ \hline 
& $0.985 & $0.985 & $0.947 & $0.971 & $0.959 \\ \hline 
& $0.987 & $0.988 & $0.953 & $0.974 & $0.964 \\ \hline 
& $0.982 & $0.981 & $0.938 & $0.971 & $0.954 \\ \hline 

